# ⚡ Anthropic SDK - Advanced Level

## Advanced Patterns and Tool Use

This notebook covers advanced features that enable powerful AI applications.

### What You'll Learn:
- Tool Use (Function Calling) - Let Claude call your functions
- Structured JSON outputs
- Prompt caching for efficiency
- Async operations for performance
- Advanced error handling and retries

---

## 1. Setup

In [ ]:
import anthropic
import json
import asyncio
from config import MODEL, ANTHROPIC_API_KEY, validate_api_key, DEFAULT_MAX_TOKENS

validate_api_key()
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

print(f"Model: {MODEL}")
print("Ready for advanced concepts!")

## 2. Tool Use (Function Calling)

Tool use allows Claude to call functions you define. This is powerful for:
- Retrieving real-time data
- Performing calculations
- Interacting with external systems
- Building AI agents

In [ ]:
# Define tools that Claude can use
tools = [
    {
        "name": "get_weather",
        "description": "Get the current weather for a location. Use this when the user asks about weather.",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state/country, e.g., 'San Francisco, CA' or 'London, UK'"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature unit (default: celsius)"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "calculate",
        "description": "Perform mathematical calculations. Use this for any math operations.",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate, e.g., '2 + 2' or 'sqrt(16)'"
                }
            },
            "required": ["expression"]
        }
    }
]

print("Tools defined:")
for tool in tools:
    print(f"  - {tool['name']}: {tool['description'][:50]}...")

In [ ]:
# Implement the actual tool functions
import math

def get_weather(location, unit="celsius"):
    """Simulated weather function - in production, call a real API."""
    # Simulated data
    weather_data = {
        "San Francisco, CA": {"temp_c": 18, "condition": "Foggy"},
        "New York, NY": {"temp_c": 22, "condition": "Sunny"},
        "London, UK": {"temp_c": 15, "condition": "Cloudy"},
    }
    
    # Default for unknown locations
    data = weather_data.get(location, {"temp_c": 20, "condition": "Clear"})
    
    temp = data["temp_c"]
    if unit == "fahrenheit":
        temp = (temp * 9/5) + 32
        
    return {
        "location": location,
        "temperature": temp,
        "unit": unit,
        "condition": data["condition"]
    }

def calculate(expression):
    """Safely evaluate mathematical expressions."""
    # Create a safe namespace with math functions
    safe_dict = {
        "abs": abs, "round": round,
        "sqrt": math.sqrt, "pow": pow,
        "sin": math.sin, "cos": math.cos, "tan": math.tan,
        "log": math.log, "log10": math.log10,
        "pi": math.pi, "e": math.e
    }
    try:
        result = eval(expression, {"__builtins__": {}}, safe_dict)
        return {"expression": expression, "result": result}
    except Exception as e:
        return {"expression": expression, "error": str(e)}

# Test the functions
print("Testing get_weather:")
print(get_weather("San Francisco, CA"))
print("\nTesting calculate:")
print(calculate("sqrt(16) + 2**3"))

In [ ]:
# Make a request with tools
response = client.messages.create(
    model=MODEL,
    max_tokens=1024,
    tools=tools,
    messages=[
        {"role": "user", "content": "What's the weather like in San Francisco?"}
    ]
)

print("Response type:", response.stop_reason)
print("\nContent blocks:")
for block in response.content:
    print(f"  Type: {block.type}")
    if block.type == "tool_use":
        print(f"  Tool: {block.name}")
        print(f"  Input: {block.input}")
        print(f"  ID: {block.id}")

In [ ]:
# Complete tool use loop - execute tool and return result
def process_tool_call(tool_name, tool_input):
    """Execute a tool and return the result."""
    if tool_name == "get_weather":
        return get_weather(**tool_input)
    elif tool_name == "calculate":
        return calculate(**tool_input)
    else:
        return {"error": f"Unknown tool: {tool_name}"}

def chat_with_tools(user_message, tools):
    """Complete conversation loop with tool use."""
    messages = [{"role": "user", "content": user_message}]
    
    while True:
        response = client.messages.create(
            model=MODEL,
            max_tokens=1024,
            tools=tools,
            messages=messages
        )
        
        # If Claude wants to use a tool
        if response.stop_reason == "tool_use":
            # Process each tool use in the response
            tool_results = []
            for block in response.content:
                if block.type == "tool_use":
                    print(f"🔧 Calling tool: {block.name}")
                    print(f"   Input: {block.input}")
                    
                    result = process_tool_call(block.name, block.input)
                    print(f"   Result: {result}\n")
                    
                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": block.id,
                        "content": json.dumps(result)
                    })
            
            # Add assistant response and tool results to messages
            messages.append({"role": "assistant", "content": response.content})
            messages.append({"role": "user", "content": tool_results})
        
        else:
            # Claude is done - return final response
            final_text = ""
            for block in response.content:
                if hasattr(block, "text"):
                    final_text += block.text
            return final_text

# Test the complete loop
print("=" * 60)
result = chat_with_tools("What's the weather in London and what's 15 * 7?", tools)
print("=" * 60)
print("\nFinal Response:")
print(result)

## 3. Structured JSON Output

Force Claude to output valid JSON for structured data extraction.

In [ ]:
# Method 1: Using system prompt and prefilled assistant message
def extract_json(text, schema_description):
    """Extract structured JSON from text."""
    response = client.messages.create(
        model=MODEL,
        max_tokens=1024,
        system=f"""You are a data extraction assistant. Extract information and return ONLY valid JSON.
Schema: {schema_description}
Return ONLY the JSON object, no other text.""",
        messages=[
            {"role": "user", "content": f"Extract data from: {text}"},
            {"role": "assistant", "content": "{"}  # Prefill to force JSON
        ]
    )
    
    # Reconstruct the JSON (we prefilled with "{")
    json_str = "{" + response.content[0].text
    return json.loads(json_str)

# Test extraction
text = """
John Smith is a 35-year-old software engineer from Seattle. 
He has 10 years of experience and specializes in Python and JavaScript.
His email is john.smith@email.com.
"""

schema = """
{
    "name": "string",
    "age": "number",
    "occupation": "string",
    "location": "string",
    "skills": ["string"],
    "email": "string"
}
"""

result = extract_json(text, schema)
print("Extracted Data:")
print(json.dumps(result, indent=2))

In [ ]:
# Method 2: Using tools for guaranteed JSON schema
def extract_with_tool(text, tool_schema):
    """Use tool use to guarantee JSON output matches schema."""
    extraction_tool = {
        "name": "extract_data",
        "description": "Extract and structure data from text",
        "input_schema": tool_schema
    }
    
    response = client.messages.create(
        model=MODEL,
        max_tokens=1024,
        tools=[extraction_tool],
        tool_choice={"type": "tool", "name": "extract_data"},  # Force tool use
        messages=[
            {"role": "user", "content": f"Extract the following information: {text}"}
        ]
    )
    
    # Get the tool input (which is our structured data)
    for block in response.content:
        if block.type == "tool_use":
            return block.input
    return None

# Define schema as JSON Schema
person_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Full name"},
        "age": {"type": "integer", "description": "Age in years"},
        "occupation": {"type": "string", "description": "Job title"},
        "skills": {
            "type": "array",
            "items": {"type": "string"},
            "description": "List of skills"
        }
    },
    "required": ["name", "age", "occupation"]
}

result = extract_with_tool(text, person_schema)
print("Extracted with Tool Schema:")
print(json.dumps(result, indent=2))

## 4. Prompt Caching

Cache long prompts to reduce latency and costs for repeated requests.

In [ ]:
# Prompt caching example
# The system prompt or large context can be cached

# Create a long context (needs to be at least 1024 tokens for caching)
long_context = """
# Python Programming Guide

## Chapter 1: Introduction to Python
Python is a high-level, interpreted programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991. Python's design philosophy 
emphasizes code readability with the use of significant indentation.

## Chapter 2: Variables and Data Types
Python supports various data types including integers, floats, strings, lists, tuples, 
dictionaries, and sets. Variables in Python are dynamically typed, meaning you don't need 
to declare their type explicitly.

## Chapter 3: Control Flow
Python uses if, elif, and else statements for conditional execution. For loops iterate over 
sequences, while loops continue until a condition is false. Python also supports break, 
continue, and pass statements.

## Chapter 4: Functions
Functions in Python are defined using the def keyword. They can accept positional arguments, 
keyword arguments, *args for variable positional arguments, and **kwargs for variable 
keyword arguments. Python supports lambda functions for simple anonymous functions.

## Chapter 5: Object-Oriented Programming
Python supports object-oriented programming with classes and objects. Classes define 
blueprints for objects, encapsulating data and behavior. Python supports inheritance, 
polymorphism, and encapsulation.

## Chapter 6: Modules and Packages
Python code can be organized into modules (single files) and packages (directories of modules).
The import statement allows you to use code from other modules. Python has a rich standard 
library and thousands of third-party packages available via pip.

## Chapter 7: Error Handling
Python uses try-except blocks for error handling. You can catch specific exceptions or use 
a general except clause. The finally block executes regardless of whether an exception occurred.
You can also raise custom exceptions.

## Chapter 8: File I/O
Python provides built-in functions for reading and writing files. The open() function returns 
a file object, and you can use context managers (with statement) to ensure proper file handling.
Python supports text and binary file modes.
""" * 3  # Repeat to ensure enough tokens

# First request - will cache the context
response1 = client.messages.create(
    model=MODEL,
    max_tokens=200,
    system=[
        {
            "type": "text",
            "text": long_context,
            "cache_control": {"type": "ephemeral"}  # Mark for caching
        }
    ],
    messages=[
        {"role": "user", "content": "What is Python according to Chapter 1?"}
    ]
)

print("First Request (caching the context):")
print(f"Input tokens: {response1.usage.input_tokens}")
if hasattr(response1.usage, 'cache_creation_input_tokens'):
    print(f"Cache creation tokens: {response1.usage.cache_creation_input_tokens}")
if hasattr(response1.usage, 'cache_read_input_tokens'):
    print(f"Cache read tokens: {response1.usage.cache_read_input_tokens}")
print(f"\nResponse: {response1.content[0].text[:200]}...")

In [ ]:
# Second request - should use cached context
response2 = client.messages.create(
    model=MODEL,
    max_tokens=200,
    system=[
        {
            "type": "text",
            "text": long_context,
            "cache_control": {"type": "ephemeral"}
        }
    ],
    messages=[
        {"role": "user", "content": "What does Chapter 5 cover?"}
    ]
)

print("Second Request (using cached context):")
print(f"Input tokens: {response2.usage.input_tokens}")
if hasattr(response2.usage, 'cache_creation_input_tokens'):
    print(f"Cache creation tokens: {response2.usage.cache_creation_input_tokens}")
if hasattr(response2.usage, 'cache_read_input_tokens'):
    print(f"Cache read tokens: {response2.usage.cache_read_input_tokens}")
print(f"\nResponse: {response2.content[0].text[:200]}...")

## 5. Async Operations

Use async for better performance when making multiple concurrent requests.

In [ ]:
import time

# Create async client
async_client = anthropic.AsyncAnthropic(api_key=ANTHROPIC_API_KEY)

async def async_chat(message):
    """Make an async API call."""
    response = await async_client.messages.create(
        model=MODEL,
        max_tokens=100,
        messages=[{"role": "user", "content": message}]
    )
    return response.content[0].text

async def run_parallel_requests():
    """Run multiple requests in parallel."""
    questions = [
        "What is 2+2?",
        "Name a color.",
        "What is Python?",
        "Name a planet.",
        "What is AI?"
    ]
    
    print("Running 5 requests in parallel...")
    start = time.time()
    
    # Run all requests concurrently
    tasks = [async_chat(q) for q in questions]
    results = await asyncio.gather(*tasks)
    
    elapsed = time.time() - start
    print(f"Completed in {elapsed:.2f} seconds\n")
    
    for q, r in zip(questions, results):
        print(f"Q: {q}")
        print(f"A: {r[:100]}...")
        print()
    
    return results

# Run the async function
# In Jupyter, we can use await directly
results = await run_parallel_requests()

In [ ]:
# Compare with sequential requests
async def run_sequential_requests():
    """Run requests sequentially for comparison."""
    questions = [
        "What is 2+2?",
        "Name a color.",
        "What is Python?",
        "Name a planet.",
        "What is AI?"
    ]
    
    print("Running 5 requests sequentially...")
    start = time.time()
    
    results = []
    for q in questions:
        result = await async_chat(q)
        results.append(result)
    
    elapsed = time.time() - start
    print(f"Completed in {elapsed:.2f} seconds")
    print("(Compare with parallel time above)")
    
    return results

sequential_results = await run_sequential_requests()

## 6. Advanced Error Handling with Retries

Implement robust error handling with exponential backoff.

In [ ]:
import random

class RobustAnthropicClient:
    """A wrapper with automatic retries and error handling."""
    
    def __init__(self, client, max_retries=3, base_delay=1.0):
        self.client = client
        self.max_retries = max_retries
        self.base_delay = base_delay
    
    def _calculate_delay(self, attempt):
        """Calculate delay with exponential backoff and jitter."""
        delay = self.base_delay * (2 ** attempt)
        jitter = random.uniform(0, 0.1 * delay)
        return delay + jitter
    
    def create_message(self, **kwargs):
        """Create a message with automatic retries."""
        last_exception = None
        
        for attempt in range(self.max_retries):
            try:
                return self.client.messages.create(**kwargs)
            
            except anthropic.RateLimitError as e:
                last_exception = e
                delay = self._calculate_delay(attempt)
                print(f"Rate limited. Retrying in {delay:.2f}s (attempt {attempt + 1}/{self.max_retries})")
                time.sleep(delay)
            
            except anthropic.APIConnectionError as e:
                last_exception = e
                delay = self._calculate_delay(attempt)
                print(f"Connection error. Retrying in {delay:.2f}s (attempt {attempt + 1}/{self.max_retries})")
                time.sleep(delay)
            
            except anthropic.InternalServerError as e:
                last_exception = e
                delay = self._calculate_delay(attempt)
                print(f"Server error. Retrying in {delay:.2f}s (attempt {attempt + 1}/{self.max_retries})")
                time.sleep(delay)
            
            except anthropic.APIStatusError as e:
                # Don't retry client errors (4xx)
                if 400 <= e.status_code < 500:
                    raise
                last_exception = e
                delay = self._calculate_delay(attempt)
                print(f"API error {e.status_code}. Retrying in {delay:.2f}s")
                time.sleep(delay)
        
        # All retries exhausted
        raise last_exception

# Create robust client
robust_client = RobustAnthropicClient(client)

# Test it
response = robust_client.create_message(
    model=MODEL,
    max_tokens=100,
    messages=[{"role": "user", "content": "Hello!"}]
)
print("Response:", response.content[0].text)

## 7. Practice Exercises